# Get injections and detection information

We want to compare the standard Bayesian results of the LVK GWTC data to a faster Fisher matrix approach (with and without adding the prior information). Therefore, we proceed as follows:
1. We need **injections**: we consider BBH events (for which we have a record of the priors that were used for the Beysian analysis) and, in particular, we take the median value from the GWTC data as the injection for `gwfish`
2. We need the list of **detectors** that detected each event and their **sensitivity curve** (since we know which detectors were operational for a given event, we do not consider the duty cycle)
3. We do not impose any SNR threshold for the Fisher analysis (even though it is customary to set a threshold of at least SNR=8 for the Fisher analysis)
4. We impose the very same **priors** as described in the LVK records

In [1]:
from tools import *
import numpy as np

List of the main folders where results are stored

In [2]:
PATH_TO_LVK_DATA = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/GWTC_LVK_data/'
PATH_TO_RESULTS = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/'
PATH_TO_YAML = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/yamls/'
PATH_TO_PSD = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/gwtc_psds/'

The waveform we choose is the `IMRPhenomXPHM` as this is the one used for BBH analysis in LVK and BBH sources are the ones in which we are interested (the majority of the detected signals)

In [3]:
waveform = 'IMRPhenomXPHM'

The function `get_events_list` goes through the GWTC data and makes list with the following properties:
- list of events that are not BBHs (4 events): *events_without_wf_IMRPhenomXPHM.txt*
- list of events that do not report the analytic priors used for the analysis (8 events): *events_without_priors_IMRPhenomXPHM.txt*
- list of events that do not report the `maxP` estimate of the parameters: *events_without_maxP_IMRPhenomXPHM.txt*
- list of events that are BBHs and have analytic priors (78 events): *events_wf_median_priors_IMRPhenomXPHM.txt*
- list of events that are BBHs and have both analytic priors and the estimate of the `maxP` value (67 events): *events_wf_maxP_priors_IMRPhenomXPHM.txt*

All events have the `median` value, easily retrieved from the posterior samples. The list of events with both the estimators, `median` and `maxP`, and analytic priors, are listed in *events_wf_median_maxP_priors_IMRPhenomXPHM.txt* (67 events)

In our anaysis we use the 78 events from *events_wf_median_priors_IMRPhenomXPHM.txt*

In [ ]:
get_events_list(PATH_TO_LVK_DATA, PATH_TO_RESULTS, waveform = 'IMRPhenomXPHM')

In [4]:
events = np.loadtxt(PATH_TO_RESULTS + 'info/' + 'events_wf_median_priors_%s.txt' %waveform, dtype=str)

Here is the list of the parameters we use for our analysis. We get the median value and use it as the injection for the Fisher analysis with `gwfish`.

In [5]:
params = ['chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 
            'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl']

In [ ]:
get_injections_from_gwtc(PATH_TO_LVK_DATA, PATH_TO_RESULTS + 'injections/', events, waveform, params, 'median')

For the complete analysis, in which we apply priors to our Fisher results, we need the same priors used in the LVK Bayesian analysis. All the parameters have fixed analytic forms for the prior, except for `chirp_mass` and `geocent_time`. This parameter has a uniform prior in a range that is event-dependent. To this scope, we store the lower and upper bounds from the LVK data in a specific dictionary.

In [ ]:
check_and_store_chirp_mass_priors(PATH_TO_LVK_DATA, PATH_TO_RESULTS, events, waveform)

The same needs to be done with the `geocent_time` lower and upper range of the prior interval

In [ ]:
check_and_store_geocent_time_priors(PATH_TO_LVK_DATA, PATH_TO_RESULTS, events, waveform)

We also collect all the information about detectors that detected a certain event and their sensitivity curve at the time of detection. All of this information is stored in a specific format in a `.yaml` file that is then passed to `gwfish` for the fisher analysis.

In [6]:
detectors_and_yaml_files(PATH_TO_LVK_DATA, PATH_TO_RESULTS, PATH_TO_YAML, PATH_TO_PSD, events, waveform)